# Что за хитрые валидации?

> 🚀 В этой практике нам понадобятся: `numpy==1.26.4, pandas==2.2.2, scikit-learn==1.4.2` 

> 🚀 Установить вы их можете с помощью команды: `%pip install numpy==1.26.4 pandas==2.2.2 scikit-learn==1.4.2` 


## Содержание

* [Выборка для валидации (Валидационная выборка - Validation set)](#Выборка-для-валидации-Валидационная-выборка---Validation-set)
* [Кросс-валидация (Cross-Validation)](#Кросс-валидация-Cross-Validation)
* [Поиск гиперпараметров](#Поиск-гиперпараметров)
* [Задание](#Задание)
* [Вопросы для закрепления](#Вопросы-для-закрепления)
* [Полезные ссылки](#Полезные-ссылки)


Привет! Сегодня мы узнаем очень важный аспект разработки модели машинного обучения в реальных условиях!

Если раньше мы почти не занимались настройкой моделей (так как параметров почти не было) и нам было достаточно разработать модель на выборке для обучения и дальше проверить работу на "новых данных" (тестовая выборка).

Но на последнем занятии мы столкнулись с кучей разных моделей и, оказывается, в разных моделях бывает много разных параметров! Мы уже задумывались о том, что использовать тестовую выборку для настройки параметров нельзя, потому что так мы проводим "неявное" обучение и настройку гиперпараметров, а значит у нас после этого отсутствуют действительно новые данные, на которых можно снять окончательные метрики.

Что же нам тогда делать? Выход есть и начнём мы с наиболее простого примера!

In [ ]:
# Настройки для визуализации
# Если используется темная тема - лучше текст сделать белым
import numpy as np
import pandas as pd
import random
import os

pd.set_option("display.max_columns", 50)

# Зафиксируем состояние случайных чисел
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)
random.seed(RANDOM_SEED)

## Выборка для валидации (Валидационная выборка - Validation set)

Валидационной выборкой называется такая выборка, которая используется для настройки *гиперпараметров* модели.

> 🤓 В свою очередь выборка для обучения используется для настройки *параметров* модели.

> ✨ Если вы успели позабыть, что означают эти два термина - обсудите с преподавателем

А как нам эту выборку получить? Тут тоже нет ничего сложного - просто после деления на train-test из обучающих данных выделяется ещё одна небольшая выборка (valid). Обычно, если данных очень много, то данные делятся в пропорции 60-20-20 (train-test-valid). Если данных не сильно много (зависит от ситуации и задачи, но в бинарной классификации можно за порог считать порядка 10000 примеров), то лучше больше выделить на тест, чтобы проверить работоспособность модели на совершенно новых невиданных ранее данных - 50-20-30.

Давайте сделаем свою функцию, которая производит деление на train-test-valid с указанием процентов соотношений:

> ⚠️ В этом примере мы отключим стратификацию по двум причинам: далее мы будем работать с большими данными без дисбаланса - случайное разделение достаточно, чтобы данные попали во все выборки, и проблема переполнения на Windows с типом по-умолчанию https://github.com/numpy/numpy/issues/9464

> ⚠️ В целом, далее рекомендуется использовать стратификацию, но проверять результат работы!

In [ ]:
# TODO - создайте функцию train_test_valid_split(), которая принимает
#   df - DataFrame
#   test_size - пропорция выборки для тестирования
#   valid_size - пропорция выборки для валидации
# Стратификацию не используем - причина выше
# Функция должна возвращать три DataFrame в порядке - train, test, valid

In [ ]:
# TEST

rng = np.random.default_rng(RANDOM_SEED)
_test_size = 1000
_test_df = pd.DataFrame({"col": rng.integers(0, 100, size=_test_size), "target": rng.choice([0, 1], size=_test_size)})

_test_result_train, _test_result_test, _test_result_valid = train_test_valid_split(_test_df, test_size=0.1, valid_size=0.3)

assert _test_result_train.shape[0] == 600
assert _test_result_test.shape[0] == 100
assert _test_result_valid.shape[0] == 300

_test_expected_vc_train = pd.Series([303, 297], index=[0, 1], name="count")
_test_expected_vc_train.index.name = "target"
pd.testing.assert_series_equal(_test_result_train["target"].value_counts(), _test_expected_vc_train)

_test_expected_vc_test = pd.Series([52, 48], index=[1, 0], name="count")
_test_expected_vc_test.index.name = "target"
pd.testing.assert_series_equal(_test_result_test["target"].value_counts(), _test_expected_vc_test)

_test_expected_vc_valid = pd.Series([156, 144], index=[0, 1], name="count")
_test_expected_vc_valid.index.name = "target"
pd.testing.assert_series_equal(_test_result_valid["target"].value_counts(), _test_expected_vc_valid)

print("Well done!")

Отлично! Молодцы! 

Для испытаний новой функции и освоения подходов подбора гиперпараметров воспользуемся датасетом [Covertype Data Set](https://www.kaggle.com/teejmahal20/airline-passenger-satisfaction).

> Выгружайте файлы `train.csv` и `test.csv` и кладите в папку `airline_dataset` рядом с ноутбуком.

In [ ]:
TRAIN_PATH = os.path.join("airline_dataset", "train.csv")
TEST_PATH = os.path.join("airline_dataset", "test.csv")

df_train = pd.read_csv(TRAIN_PATH, sep=",", index_col=0)
df_test = pd.read_csv(TEST_PATH, sep=",", index_col=0)

df = pd.concat((df_train, df_test), ignore_index=True)
# df.reset_index(inplace=True, drop=True)

# Определим полезные константы и переменные
TARGET_COLUMN = "satisfaction"

x_columns = df_train.columns
x_columns = x_columns[x_columns != TARGET_COLUMN]

Сейчас мы слили файлы воедино, так как наша функция умеет сама разделять с нужной пропорцией на выборки.

> При участии в соревнованиях нельзя использовать тестовую выборку для обучения. Нужно валидацию брать от выборки обучения.

Давайте бегло посмотрим на данные

In [ ]:
df.shape

In [ ]:
df.head(10)

In [ ]:
df.isna().sum()

In [ ]:
df.nunique(dropna=False)

Как видите, в датасете мы имеем колонку `id`, которая имеет только уникальные числовые значения и колонку `Arrival Delay in Minutes`, которая имеет пропуски.

Так как колонка числовая, то заполнение выполним с помощью SimpleImputer, а колонку `id` просто удалим из датасета.

Также, нам нужно закодировать колонки `Gender`, `Customer Type`, `Type of Travel`, `Class`, так как они имеют строковые значения. Помимо этого нам надо ещё и целевую закодировать!

Уф, столько дел!

Давайте напишем свой класс для предобработки данных!

In [ ]:
df_train, df_test, df_valid = train_test_valid_split(df, test_size=0.2, valid_size=0.2)

# Вот эта проверка очень важна! Вы можете сами убедиться и попробовать разделить датасет стандартной функцией train_test_split() со стратификацией по целевой колонке
# В результате подобная проверка не пройдет
assert len(df) == len(df_train) + len(df_test) + len(df_valid)

df_train.shape, df_test.shape, df_valid.shape

In [ ]:
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.impute import SimpleImputer


class DataPreprocess:
    def __init__(self):
        self._imputer = SimpleImputer(strategy="median")
        self._impute_cols = ["Arrival Delay in Minutes"]

        self._encoder = OneHotEncoder(sparse_output=False)
        self._encode_cols = ["Gender", "Customer Type", "Type of Travel", "Class"]

        self._label_enc = LabelEncoder()

    def fit(self, X, y):
        self._imputer.fit(X[self._impute_cols])
        self._encoder.fit(X[self._encode_cols])
        self._label_enc.fit(y)

    def transform(self, X, y):
        X = X.copy()
        X.drop(columns=["id"], inplace=True)

        X[self._impute_cols] = self._imputer.transform(X[self._impute_cols])
        encoded_data = self._encoder.transform(X[self._encode_cols])
        new_col_names = self._encoder.get_feature_names_out(self._encode_cols)

        X_enc = pd.DataFrame(data=encoded_data, columns=new_col_names, index=X.index)

        X.drop(columns=self._encode_cols, inplace=True)
        X = pd.concat([X, X_enc], axis=1)

        y = self._label_enc.transform(y)

        return X, y

Теперь давайте применим и обучим модель случайного леса на наших данных и посмотрим метрики!

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

rf_clf = RandomForestClassifier(max_depth=1, n_estimators=100, random_state=RANDOM_SEED)
data_prep = DataPreprocess()

y_train = df_train[TARGET_COLUMN]
x_train = df_train[x_columns]

y_test = df_test[TARGET_COLUMN]
x_test = df_test[x_columns]


data_prep.fit(x_train, y_train)
x_train_enc, y_train_enc = data_prep.transform(x_train, y_train)
rf_clf.fit(x_train_enc, y_train_enc)

In [ ]:
x_test_enc, y_test_enc = data_prep.transform(x_test, y_test)
y_pred = rf_clf.predict(x_test_enc)

report = classification_report(y_test_enc, y_pred)
print(report)

Отлично, вот мы обучили модель с какими-то параметрами, но что если мы хотим попробовать другие параметры? Нам нужно в таком случае использовать выборку для валидации. Давайте попробуем перебрать пару параметров в цикле:

In [ ]:
from sklearn.metrics import f1_score

y_train = df_train[TARGET_COLUMN]
x_train = df_train[x_columns]

y_valid = df_valid[TARGET_COLUMN]
x_valid = df_valid[x_columns]

data_prep = DataPreprocess()

data_prep.fit(x_train, y_train)
x_train_enc, y_train_enc = data_prep.transform(x_train, y_train)
x_valid_enc, y_valid_enc = data_prep.transform(x_valid, y_valid)

for max_depth_value in np.arange(3, 6):
    for n_estimators_value in np.arange(70, 110, step=10):
        print(f"Try max_depth: {max_depth_value} | n_estimators: {n_estimators_value}")
        
        rf_clf = RandomForestClassifier(max_depth=max_depth_value, n_estimators=n_estimators_value, random_state=RANDOM_SEED)
        rf_clf.fit(x_train_enc, y_train_enc)

        y_val_pred = rf_clf.predict(x_valid_enc)

        f1_value = f1_score(y_valid_enc, y_val_pred, average="macro")

        print(f"F1 score: {f1_value}")

Смотрите, мы пробежались по паре параметров и выявили наилучший вариант из возможных заданных! Отлично, таким образом, мы можем взять лучшую пару параметров и улучшить показания на тестовой выборке:

In [ ]:
rf_clf = RandomForestClassifier(max_depth=5, n_estimators=90, random_state=RANDOM_SEED)
rf_clf.fit(x_train_enc, y_train_enc)

y_pred = rf_clf.predict(x_test_enc)

report = classification_report(y_test_enc, y_pred)
print(report)

Замечательно! Как видите, мы действительно улучшили результат, а никакого переобучения не наблюдается из-за **подгона гиперпараметров** под данные. Вот так удобно можно использовать валидационную выборку!

## Кросс-валидация (Cross-Validation)

Всё было бы хорошо, но есть одна важная проблема - данных не всегда бывает много!

Иногда, нам приходится обучать модели на данных размеров менее 10000 примеров, что уже накладывает ограничение на разделение. Откусить 30%, чтобы тест был более-менее репрезентативным, по-хорошему на валидацию надо процентов 20, что тогда остаётся - всего пол датасета на обучение? Ну, дела!

Для таких случаев, умные люди придумали [кросс-валидацию](https://scikit-learn.org/stable/modules/cross_validation.html). Это процесс, при котором не требуется отдельно брать валидационную выборку. Мы просто используем часть набора для обучения для валидации.

Но стоп, разве мы не делали именно то же самое? От выборки для обучения откусывали кусочек и фиксировали для оценки того, какие показатели при нынешних гиперпараметрах??

Не всё так просто! Мы не просто откусили и положили, а делаем это по-очереди несколько раз, а именно K раз. Такой процесс зовется **K-fold кросс-валидацией**. Гляньте картинку:

<p align="center"><img src="https://raw.githubusercontent.com/AleksDevEdu/ml_edu/master/assets/32_grid_search.png" width=800/></з>

По сути, процесс кросс-валидации заключается в следующем:
- Делаем из выборки для обучения K фолдов, например, 5
- На первой итерации берём первый фолд как валидационный, а на остальных учим модель (4 фолда)
- Оцениваем параметры по первому фолду, записываем
- На второй итерации - второй фолд валидационный, а остальные 4 - обучения
- Также, оценили, записали
- После всех K итераций усредняем оценки и получаем результирующую метрику на валидации!

Вуаля!

Что мы получаем в итоге? В K раз больше операций, но зато в ходе кросс-валидации мы всецело провалидировали модель на всем наборе данных для обучения без необходимости отдельно выделять ЕЩЁ данных из и так малого количества! 

Давайте посмотрим, как это работает!

In [ ]:
# Соединим выборки обучения и валидации так как при кросс-валидации на отдельная выборка не нужна
# Тестовая остается та же, чтобы было корреткное сравнение
df_train_valid = pd.concat((df_train, df_valid), ignore_index=True)
df_train_valid.shape

> Следующий блок кода может занять до пяти минут. Просим проявить терпение, ведь обучение - совсем не быстрый процесс.

In [ ]:
from sklearn.model_selection import cross_val_score

y_train = df_train_valid[TARGET_COLUMN]
x_train = df_train_valid[x_columns]

data_prep = DataPreprocess()

data_prep.fit(x_train, y_train)
x_train_enc, y_train_enc = data_prep.transform(x_train, y_train)

for max_depth_value in np.arange(3, 6):
    for n_estimators_value in np.arange(70, 110, step=10):
        print(f"Try max_depth: {max_depth_value} | n_estimators: {n_estimators_value}")
        
        rf_clf = RandomForestClassifier(max_depth=max_depth_value, n_estimators=n_estimators_value, random_state=RANDOM_SEED)

        f1_values = cross_val_score(estimator=rf_clf, X=x_train_enc, y=y_train_enc, cv=5, scoring="f1_macro")
        print(f"F1 score: {f1_values}, mean: {np.mean(f1_values)}")

Вот так просто можно получить рабочий вариант валидации на малом датасете! Вообще не сложно, правда?

## Поиск гиперпараметров

А теперь представьте, что в модели не два, а куууча параметров. Вообще, если обратиться к странице [RandomForestClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html), то вы увидите, что параметров там больше, чем два! И что теперь, писать кучу вложенных циклов? Вааау, получится примерно так:

<p align="center"><img src="https://raw.githubusercontent.com/AleksDevEdu/ml_edu/master/assets/indent_meme.jpg" width=800/></p>

А чего тогда делать?? Есть для вас интересный вариант! Посмотрим на [GridSearchCV](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html).

По сути, это класс, которому мы отдаём нашу модель, список параметров, которые надо перебрать и после этого можно спокойно ожидать результата. Работает это так:

> И этот блок кода тоже может занять до пяти минут =)

In [ ]:
from sklearn.model_selection import GridSearchCV

y_train = df_train_valid[TARGET_COLUMN]
x_train = df_train_valid[x_columns]

data_prep = DataPreprocess()

data_prep.fit(x_train, y_train)
x_train_enc, y_train_enc = data_prep.transform(x_train, y_train)

rf_clf = RandomForestClassifier(random_state=RANDOM_SEED)
grid_search = GridSearchCV(
    estimator=rf_clf,
    param_grid={
        "max_depth": np.arange(3, 6),
        "n_estimators": np.arange(70, 110, step=10),
    },
    scoring="f1_macro",
    cv=5
)

grid_search.fit(x_train_enc, y_train_enc)

Да, GridSearch работает по такому же интерфейсу, как и любая модель, но важно отметить, что внутри него содержится масса полезных атрибутов! Давайте на них посмотрим!

In [ ]:
# Можно посмотреть сетку, которую перебирал GS
grid_search.cv_results_["params"]

In [ ]:
# Скор лучшего набора данных
grid_search.best_score_

In [ ]:
# Лучшие параметры
grid_search.best_params_

In [ ]:
# И даже сразу лучшую обученную модель с лучшими параметрами
grid_search.best_estimator_

По сути, интерфейс GridSearch позволяет выполнять автоматический поиск гиперпараметров методом кросс-валидации, перебирая все возможные комбинации из заданных параметров.

> ⚠️ Существует также и альтернативный способ поиска, который не ограничивается на исключительном переборе, а основывается на вероятностном выборе из распределений. Вы задаете конкретные диапазоны, а метод случайно выбирает значения и испытывает их. Поиск завершается, когда истекло заданное количество итераций, так как при работе с распределениями можно бесконечно делать выборки значений. Вот это метод - [RandomizedSearchCV](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.RandomizedSearchCV.html).

Вот так мы познакомились с полезным инструментом для автоматизированного поиска гиперпараметров модели. Такая возможность позволяет искать гиперпараметры без каких-либо сложностей!

## Задание

Сегодня мы научились важному методу поиска лучших гиперпараметров, так как их выбор может сильно сказаться на результатах работы с данными.

Все сегодняшние инструменты позволят вам более гибко и уверенно производить разработку моделей для решения поставленных задач, что делает вас ещё круче, чем вы были раньше. Гиперпараметры моделей теперь вам ни по чём! Круто - поздравляем!

Но не оставлять же вас без веселья? Как видите, у нас на руках датасет с очень внушительным объёмом - попробуйте решить поставленную на нём задачу, найдя наиболее подходящую модель и её гиперпараметры!

* Задачка под зведочкой - если вы проведёте анализ данных и дополнительные обработки, то, вероятно, вам удастся улучшить результаты работы модели ещё сильнее! Дерзайте! 

## Вопросы для закрепления

А теперь пара вопросов, чтобы закрепить материал!

1. Почему кросс-валидацию не всегда выгодно использовать? В каких ситуациях?
2. Нужно ли задавать отдельную валидационную выборку для кросс-валидации? 
3. Зачем нужен подбор гиперпараметров модели? Разве это не читерство? 
4. Выгодно ли с точки зрения временных рамок указывать много разных параметров для GridSearchCV? 
5. В чём разница между GridSearchCV и RandomizeSearchCV?

## Полезные ссылки
* [Cross Validation от StatQuest](https://www.youtube.com/watch?v=fSytzGwwBVw)
